In [1]:
import nltk
import matplotlib.pyplot as plt
from nltk.corpus import sentiwordnet as swn
import math
from math import exp, expm1, log, log10
import numpy as np
import turtle
import pandas as pd
from nltk.wsd import lesk
from pycorenlp import StanfordCoreNLP
from pywsd.lesk import simple_lesk
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
import sys, os


nlp = StanfordCoreNLP('http://localhost:9000')
dependency_parser = nlp.annotate
# positive_lexicon = []
# negative_lexicon = []

# def read_lexicon():
#     global positive_lexicon;
#     global negative_lexicon;
    
#     with open(os.path.join(os.path.abspath('../opinion-lexicon-English/') , 'positive-words.txt'), 'r') as file:
#         line = file.readline();
#         while ";" in line:
#             line = file.readline();
         
#         positive_lexicon = file.readlines()
    
#     with open(os.path.join(os.path.abspath('../opinion-lexicon-English/') , 'negative-words.txt'), 'r', encoding = "ISO-8859-1") as file:
#         line = file.readline();
#         while ";" in line:
#             line = file.readline();
        
#         negative_lexicon = file.readlines()
        
#     positive_lexicon = list(map(lambda word: word.rstrip("\n\r"), positive_lexicon))
#     negative_lexicon = list(map(lambda word: word.rstrip("\n\r"), negative_lexicon))
    
        
# read_lexicon()
# op_set = positive_lexicon + negative_lexicon

negation = [
    "afraid",
    "can't",
    "cannot",
    "deny",
    "mean",
    "negate",
    "negation",
    "negative",
    "neither",
    "never",
    "no",
    "non",
    "none",
    "nor",
    "not",
    "nothing",
    "refusal",
    "refuse",
    "reject",
    "rejection"
]

def analyse_file(key, lines):    
    radii = get_TDOC(lines, key)    
    return radii

def get_TDOC(lines, key):
    freq = {'Init': 0}              #Number of times context term occurs with key
    freq.clear()
    prohib = ['']#stopWords
    for line in lines:
        words = line.split(" ")
        if key in words:
            for context in words:
                flag=0
                for i in prohib:
                    if i == context:
                        flag=1
                        break
                if flag==0 and context!=key:# and context in op_set:
                    freq.setdefault(context, 0)
                    freq[context] = freq.get(context) + 1
                                           
    N = 0                           #Total Number of terms in Document
    for line in lines:
        words = line.split(" ")
        N += len(words)

    Nci = {'Init': 0}               #Total terms that occur with context term
    Nci.clear()
    for context in freq.keys():
        for line in lines:
            words = line.split(" ")
            if context in words:
                Nci.setdefault(context, 0)
                Nci[context] += len(words)

    radii = {'Init': 0}             #Get Radius of context term with TDOC formula
    radii.clear()
    
    df = pd.DataFrame(columns=['c', 'm', 'N', 'Nc', 'f', 'N/Nc', 'log(N/Nc)', 'fxlog(N/Nc)', '/4'])
    max_value = 0
    for term in freq.keys():
        radii[term] = (freq[term]*(log(N/Nci[term])))
        
        if radii[term] > max_value:
            max_value = radii[term]
        
    for term in freq.keys():
        radii[term] = radii[term]/max_value
        
        df = df.append({'c': term,
                'm': key,
                'N': N,
                'Nc': Nci[term],
                'f': freq[term],
                'N/Nc': "{0:.2f}".format(N/Nci[term]),
                'log(N/Nc)': "{0:.2f}".format(log(N/Nci[term])),
                'fxlog(N/Nc)': "{0:.2f}".format(freq[term]*(log(N/Nci[term]))),
                'normalisasi': "{0:.2f}".format((freq[term]*(log(N/Nci[term])))/max_value)
               }, ignore_index=True)
    
#     df.to_excel("tdoc2.xlsx")
    return radii                    #Returns entire set of context terms related to key

def pos_tag(sentence):
    result = dependency_parser(sentence, properties={"outputFormat": "json", "annotators": "pos"})['sentences'][0]['tokens']
    res = []
    for pos in result:
        res.append((pos['word'], pos['pos']))
    return res

def get_theta(key, sentences):
    scores = []
    for sentence in sentences:
        flag = True
        
        pp_tagged = pos_tag(sentence)
        tagged = ('','')
        for p in pp_tagged:
            if p[0] == key:
                tagged = p
        if tagged == ('', ''):
            flag = False
        
        ambiguous = tagged[0]
        tag = tagged[1]
        pos = ''

        if 'NN' in tag:
            pos = 'n'
        elif 'NNS' in tag:
            pos = 'nns'
        elif 'VB' in tag:
            pos = 'v'
        elif 'VBG' in tag:
            pos = 'v'
        elif 'JJ' in tag:
            pos = 'a'
        elif 'RB' in tag:
            pos = 'r'
        else:
            flag = False

        if flag:
            answer = simple_lesk(sentence, ambiguous, pos)
            if answer:
                score = swn.senti_synset(answer.name())
                endscore = 0
                
                if score.pos_score() > score.neg_score():
                    endscore = score.pos_score()
                else:
                    endscore = score.neg_score() * (-1)
                
                words = sentence.split(' ')
                word_around = []
                for x in range(0, len(words)):
                    try:
                        if (words[x+1] == key) or (words[x+2] == key) or (words[x+3]== key):
                            word_around.append(words[x])
                        elif (words[x-1] == key) or (words[x-2] == key) or (words[x-3]== key):
                            word_around.append(words[x])
                    except:
                        pass
                    
                for neg in negation:
                    if neg in word_around:
                        endscore *= (-1)
                        break
                        
                scores.append(endscore)
            else:
                scores.append(0)
        else:
            scores.append(0)
            
    final_score = np.average(scores)
    return np.pi * final_score

def prior_sentiment(radii, key, all_sentences):
    theta = {'Init': 0}
    theta.clear()
    for word in radii.keys():
        sentences = []
        for sentence in all_sentences:
            words = sentence.split(' ')
            if (word in words) and (key in words):
                sentences.append(sentence)
                
        filter = get_theta(word, sentences)            #if function returns 0 word does not exist in lexicon
        theta[word] = filter
        
    return theta

def senti(key, lines):
    radii = analyse_file(key, lines)
    theta = prior_sentiment(radii, key, lines)
    return theta

Warming up PyWSD (takes ~10 secs)... took 5.22561240196228 secs.


In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv("nokia.csv")

In [3]:
def target(data):
    index = 0
    aspect = {}
    for aspects in data:
        index += 1
        if aspects is not np.nan:
            for aspect_pair in aspects.split('|'):
                temp = aspect_pair.split('!')
                aspek = temp[0]
                try:
                    senti = temp[1]
                except:
                    senti = ''
                try:
                    aspect[aspek]
                    if senti == '+':
                        aspect[aspek]['positive'] += 1
                    elif senti == '-':
                        aspect[aspek]['negative'] += 1
                except:
                    if senti == '+':
                        aspect[aspek] = {'positive': 1, 'negative': 0}
                    elif senti == '-':
                        aspect[aspek] = {'positive': 0, 'negative': 1}
                    else:
                        aspect[aspek] = {'positive': 0, 'negative': 0}
    return aspect

In [4]:
def aspect(data):
    index = 0
    aspect = {}
    for aspects in data:
        index += 1
        if aspects is not np.nan:
            for aspect_pair in aspects.split('|'):
                if aspect_pair != '':
                    temp = aspect_pair.split('!')
                    aspek = temp[0]
                    opini = temp[1]
                    try:
                        aspect[aspek] += '|' + opini
                    except:
                        aspect[aspek] = opini
    return aspect

In [5]:
targets = target(data['aspect'])

In [6]:
aspects = aspect(data['prediction'])

In [20]:
def predict(aspects, targets, data):
    aspect = {}
    for aspek in aspects:
        if aspek in targets:
            theta = senti(aspek, data)
            for opini in aspects[aspek].split('|'):
                try:
                    aspect[aspek]
                    if theta[opini] > 0:
                        aspect[aspek]['positive'] += 1
                    else:#if theta[opini] < 0:
                        aspect[aspek]['negative'] += 1
                except:
                    if theta[opini] > 0:
                        aspect[aspek] = {'positive': 1, 'negative': 0}
                    else:#if theta[opini] < 0:
                        aspect[aspek] = {'positive': 0, 'negative': 1}
    return aspect

In [21]:
def predict_wordnet(aspects, targets, data):
    aspect = {}
    for aspek in aspects:
        if aspek in targets:
            for opini in aspects[aspek].split('|'):
                try:
                    score = swn.senti_synset(opini + '.a.1')
                    p_s = score.pos_score()
                    n_s = score.neg_score()
                except:
                    p_s = 0
                    n_s = 1
                try:
                    aspect[aspek]
                    if p_s >= n_s:
                        aspect[aspek]['positive'] += 1
                    else:
                        aspect[aspek]['negative'] += 1
                except:
                    if p_s >= n_s:
                        aspect[aspek] = {'positive': 1, 'negative': 0}
                    else:
                        aspect[aspek] = {'positive': 0, 'negative': 1}
    return aspect

In [22]:
def calculate(targets, predictions):
    t = 0
    f = 0
    total_p = 0
    total_t = 0
    
    for predict in predictions:
        t_pos = targets[predict]['positive']
        t_neg = targets[predict]['negative']

        p_pos = predictions[predict]['positive']
        p_neg = predictions[predict]['negative']
        
#         print(predict)
#         print(targets[predict])
#         print(predictions[predict])
#         print()
        
        if p_pos >= t_pos:
            t += t_pos
            f += (p_pos - t_pos)
        else:
            t += p_pos
        
        if p_neg >= t_neg:
            t += t_neg
            f += (p_neg - t_neg)
        else:
            t += p_neg
            
        total_p += (p_pos + p_neg)
        total_t += (t_pos + t_neg)
#     print(total_t, total_p)
    akurasi = t/total_p
    return t, f, akurasi

In [23]:
prediction = predict(aspects, targets, data['review'])

In [24]:
prediction_w = predict_wordnet(aspects, targets, data['review'])

In [25]:
calculate(targets, prediction) #senticircle

(138, 194, 0.41566265060240964)

In [26]:
calculate(targets, prediction_w) #sentiwordnet

(149, 183, 0.44879518072289154)